In [2]:
import json

In [3]:
with open("/Users/ndushay/jupyter-notebooks/discovery reports/discovery_report_2errors_outof_6.json", 'r') as f:
    data = json.load(f)

In [16]:
print(data)

{'rows': [{'druid': 'druid:rx212yq2993', 'errors': {}, 'counts': {'total_size': 1530211, 'mimetypes': {'image/jpeg': 2}, 'filename_no_extension': 0}}, {'druid': 'druid:xk191tr8945', 'errors': {'empty_object': True, 'missing_files': True}, 'counts': {'total_size': 0, 'mimetypes': {}, 'filename_no_extension': 0}}, {'druid': 'druid:sz766yw0944', 'errors': {}, 'counts': {'total_size': 1448949, 'mimetypes': {'image/jpeg': 2}, 'filename_no_extension': 0}}, {'druid': 'druid:sh143xn6642', 'errors': {}, 'counts': {'total_size': 3308587, 'mimetypes': {'image/jpeg': 2}, 'filename_no_extension': 0}}, {'druid': 'druid:xg322sp9596', 'errors': {'dupes': True}, 'counts': {'total_size': 6953821, 'mimetypes': {'image/jpeg': 6}, 'filename_no_extension': 0}}, {'druid': 'druid:gn012km8355', 'errors': {}, 'counts': {'total_size': 2975415, 'mimetypes': {'image/jpeg': 2}, 'filename_no_extension': 0}}], 'summary': {'objects_with_error': 2, 'mimetypes': {'image/jpeg': 1320}, 'start_time': '2021-10-18 12:03:45 -

In [10]:
summary = data.pop('summary')

In [11]:
summary

{'objects_with_error': 2,
 'mimetypes': {'image/jpeg': 1320},
 'start_time': '2021-10-18 12:03:45 -0700',
 'total_size': 1108060670}

In [12]:
rows = data.pop('rows')

In [13]:
rows

[{'druid': 'druid:rx212yq2993',
  'errors': {},
  'counts': {'total_size': 1530211,
   'mimetypes': {'image/jpeg': 2},
   'filename_no_extension': 0}},
 {'druid': 'druid:xk191tr8945',
  'errors': {'empty_object': True, 'missing_files': True},
  'counts': {'total_size': 0, 'mimetypes': {}, 'filename_no_extension': 0}},
 {'druid': 'druid:sz766yw0944',
  'errors': {},
  'counts': {'total_size': 1448949,
   'mimetypes': {'image/jpeg': 2},
   'filename_no_extension': 0}},
 {'druid': 'druid:sh143xn6642',
  'errors': {},
  'counts': {'total_size': 3308587,
   'mimetypes': {'image/jpeg': 2},
   'filename_no_extension': 0}},
 {'druid': 'druid:xg322sp9596',
  'errors': {'dupes': True},
  'counts': {'total_size': 6953821,
   'mimetypes': {'image/jpeg': 6},
   'filename_no_extension': 0}},
 {'druid': 'druid:gn012km8355',
  'errors': {},
  'counts': {'total_size': 2975415,
   'mimetypes': {'image/jpeg': 2},
   'filename_no_extension': 0}}]

In [14]:
data

{}

In [ ]:
import pandas as pd

In [9]:
df = pd.DataFrame(data)

ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.

In [54]:
df = pd.read_json('/Users/ndushay/jupyter-notebooks/discovery reports/discovery_report_errors_2of6_no_summary.json')

In [50]:
print(df)

                                                rows
0  {'druid': 'druid:rx212yq2993', 'errors': {}, '...
1  {'druid': 'druid:xk191tr8945', 'errors': {'emp...
2  {'druid': 'druid:sz766yw0944', 'errors': {}, '...
3  {'druid': 'druid:sh143xn6642', 'errors': {}, '...
4  {'druid': 'druid:xg322sp9596', 'errors': {'dup...
5  {'druid': 'druid:gn012km8355', 'errors': {}, '...


In [51]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   rows    6 non-null      object
dtypes: object(1)
memory usage: 176.0+ bytes
None


In [115]:
# Flatten data
df_nested_list = pd.json_normalize(data, 'rows')

In [116]:
print(df_nested_list)

               druid  counts.total_size  counts.mimetypes.image/jpeg  \
0  druid:rx212yq2993            1530211                          2.0   
1  druid:xk191tr8945                  0                          NaN   
2  druid:sz766yw0944            1448949                          2.0   
3  druid:sh143xn6642            3308587                          2.0   
4  druid:xg322sp9596            6953821                          6.0   
5  druid:gn012km8355            2975415                          2.0   

   counts.filename_no_extension errors.empty_object errors.missing_files  \
0                             0                 NaN                  NaN   
1                             0                True                 True   
2                             0                 NaN                  NaN   
3                             0                 NaN                  NaN   
4                             0                 NaN                  NaN   
5                             0        

In [117]:
print(df_nested_list.iloc[0:6, [0, 4, 5, 6]])

               druid errors.empty_object errors.missing_files errors.dupes
0  druid:rx212yq2993                 NaN                  NaN          NaN
1  druid:xk191tr8945                True                 True          NaN
2  druid:sz766yw0944                 NaN                  NaN          NaN
3  druid:sh143xn6642                 NaN                  NaN          NaN
4  druid:xg322sp9596                 NaN                  NaN         True
5  druid:gn012km8355                 NaN                  NaN          NaN


In [118]:
df_simpler = df_nested_list.iloc[0:6, [0, 4, 5, 6]]

In [146]:
df_simpler.loc[lambda df_simpler: df_simpler['errors.empty_object']==True].head()

,druid,errors.empty_object,errors.missing_files,errors.dupes
1,druid:xk191tr8945,True,True,NaN
